# Biblioteca

In [1]:
# !pip install google-cloud-speech
# !pip install pytube moviepy pydub
# !pip install python-Levenshtein
# !pip install jiwer
!pip install langchain
!pip install langchain openai unstructured chromadb tiktoken
!pip install langchain-openai

In [ ]:
import pandas as pd
from pytube import YouTube
from pydub import AudioSegment
from google.cloud import speech_v1p1beta1 as speech
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credenciais2.json"

import math

import Levenshtein
import jiwer

from langchain_openai import ChatOpenAI


# Funções

In [ ]:
def download_audio_from_youtube(url, output_path="audio.wav"):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).first()
    download_path = audio_stream.download(filename="temp_audio")

    audio = AudioSegment.from_file(download_path)
    audio = audio.set_frame_rate(16000).set_channels(1)
    audio.export(output_path, format="wav")

    os.remove(download_path)
    return output_path

def split_audio(audio_file, segment_length=50):
    audio = AudioSegment.from_wav(audio_file)
    segments = []
    for i in range(0, len(audio), segment_length * 1000):
        segment = audio[i:i + segment_length * 1000]
        segment_file = f"segment_{i // 1000}.wav"
        segment.export(segment_file, format="wav")
        segments.append(segment_file)
    return segments, len(audio)

def transcribe_audio_with_api_key(audio_file, api_key):
    client = speech.SpeechClient()

    with open(audio_file, "rb") as f:
        content = f.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="pt-BR",
    )

    response = client.recognize(config=config, audio=audio)

    transcripts = []
    for result in response.results:
        transcripts.append(result.alternatives[0].transcript)

    return transcripts

def transcrever(youtube_url):
    api_key = "APIKEY"

    audio_file = download_audio_from_youtube(youtube_url, "audio.wav")
    segments, audio_length = split_audio(audio_file)

    full_transcript = []
    for segment in segments:
        result = transcribe_audio_with_api_key(segment, api_key)
        full_transcript.extend(result)
        os.remove(segment)
    return full_transcript, audio_length

def transcrever_calcular(ref_text):
  transcript, audio_length = transcrever(link)
  full_transcript = ''
  for f in transcript:
    full_transcript += str(f)

  hypothesis = full_transcript
  reference = ref_text

  # wer_value = wer(reference, hypothesis)

  return (full_transcript, audio_length)


def wer(reference, hypothesis):
    """
    Calculate the Word Error Rate (WER).

    :param reference: The ground truth text (string)
    :param hypothesis: The transcribed text (string)
    :return: WER as a float
    """
    # Split the reference and hypothesis into words
    ref_words = reference.split()
    hyp_words = hypothesis.split()

    # Calculate the Levenshtein distance between the two lists of words
    distance = Levenshtein.distance(reference, hypothesis)

    # WER is the distance divided by the number of words in the reference
    wer_value = distance / max(len(reference.split()), len(hypothesis.split()), 1)  # Avoid division by zero
    return wer_value

# Transcrição de Áudio

In [ ]:
url_text_dict = {"https://www.youtube.com/watch?v=-BsAJyoj_Kw": "Boa tarde, Seu Carlos. Boa tarde, como vai? Bem. Seu Carlos, meu nome é Denise, eu sou interna do hospital e vou conversar com o senhor hoje. Ok. Qual a sua idade? 52 anos. O senhor é casado, solteiro? Casado. Trabalha com quê? Num escritório de engenharia. Nasceu aqui mesmo em Ribeirão? Nasci em Ribeirão Preto. E mora aqui? Moro aqui. Bom, Seu Carlos, o senhor está sentindo dor agora? No momento não. E me conta como é a dor que o senhor já sentiu. Eu estava trabalhando no escritório e senti essa dor, uma dor fraca, não dei muita importância. A segunda vez foi quando eu estava caminhando no final da tarde, uma dor um pouco mais forte, e em seguida diminui o ritmo da caminhada. A terceira vez foi à noite, quando eu estava em casa, há uns três dias atrás, uma dor mais fraca, mas já fiquei mais preocupado. E em que lugar do peito a dor é mais forte? No meio do peito. E a dor espalha para algum lugar? Às vezes sim, porque ela vem até o pescoço. Junto com a dor, o senhor sente algum problema, tipo falta de ar, enjoo ou batedeira no peito? Não, enjoo não. Só dessa segunda vez que a dor ficou mais forte que eu senti, somente essa vez. E o senhor tem boa digestão? Tenho. O intestino funciona bem? Funciona bem. E, Seu Carlos, você usou algum remédio para o alívio dessa dor? Não, alguns minutos após, ela passou sozinha. E alguém da sua família já teve essa dor ou teve algum problema do coração tipo um infarto? Não, ninguém da minha família, só o meu pai que é hipertenso. E o senhor fuma? Já fumei dos meus 14 aos 38 anos. Quantos cigarros por dia? Uma média de um maço, um maço e meio por dia. E o senhor bebe bebida alcoólica? Somente aos finais de semana, cerveja. Faz alguma atividade física? Não, só três a quatro vezes por semana eu faço caminhada, não pratico nenhum tipo de esporte. E o senhor se considera estressado, nervoso, ansioso? Não, não me considero, apesar de alguns problemas no escritório, mas não me considero estressado e nem nervoso. Bom, Seu Carlos, vai ser necessário te examinar e pedir alguns exames para entender melhor a situação. Ok. Aparentemente não é nada grave e o senhor pode ficar tranquilo. Vou pedir para o senhor aguardar um minuto. A enfermeira virá buscá-lo para outro lugar onde eu vou examinar. Com licença.",
 "https://www.youtube.com/watch?v=xNnPFvhLm6I": "Bom dia. Bom dia. Bom dia. Prévia para a câmera, por favor. Bom dia, eu sou Leandro, candidato à revalidação. O senhor tem 1 minuto para ler a sua atividade. Você está na UPA e recebe um paciente do sexo masculino, 45 anos, com queixa de dor torácica após aborrecimento no trabalho. Nos próximos 10 minutos, realize: anamnese direcionada, exame físico, hipótese diagnóstica, conduta terapêutica. O senhor tem 10 minutos. O senhor entendeu sua situação? Entendi. O senhor tem 10 minutos para realizá-la. Ok. Bom dia, como é o nome do senhor? Seu Antônio. Tudo bom, Seu Antônio? Pode sentar. Eu sou o Doutor Leandro, vou atender o senhor aqui na UPA hoje, tá bem? Tudo bem. Seu Antônio, no que é que eu posso te ajudar? Estou com muita dor no peito, muita dor no peito mesmo. Muita dor no peito, né, Seu Antônio? Essa dor, Seu Antônio, começou tem quanto tempo? Faz uns 40 minutos. Uns 40 minutos. Seu Antônio, essa dor numa escala de 0 a 10... É 10. 10, entendi. Essa dor é a primeira vez que você sente ela? É. Você nunca tinha sentido alguma dor como essa? Não. É uma dor como queimação, pontada? Apertando com força aqui no peito. Bem, entendi. Essa dor fica só aí no peito ou ela vai para mais algum lugar? Assim, no pescoço, aqui também. Entendi, ela fica nessa região do pescoço e o braço, e sobe só para o braço esquerdo ou para o direito também? Só para esse. Entendi. O senhor tem algum problema de saúde, Senhor Antônio? Eu não tenho, não. Acho que só pressão alta. Pressão alta, né. O senhor toma algum remédio para pressão? Tomo um remédio pequenininho. Pequenininho. Essa dor começou depois de que, Seu Antônio? Depois de uma raiva que eu tive lá no trabalho. Entendi. E o senhor teve algum tipo de discussão? Né, porque tá doendo, tá doendo demais, digo ao senhor. Entendi, entendi. E o senhor teve algum outro sintoma fora a dor? Vômito, desmaiou? Não. Fiquei um pouquinho enjoado, mas não chegou a vomitar, não, né? Mesmo com muita dor. Entendi. Além da pressão, o senhor tem mais algum problema de saúde? Nada. Qual o remédio que o senhor toma, o senhor lembra ou não? A pressão do senhor vinha controlando? Às vezes, de vez em quando. Entendi, não toma regularmente, né? Entendi. 'Ok o senhor. Meu Deus do céu. Ô, senhor, me autoriza a realizar exame físico do senhor? Autorizo. Vou precisar fazer exame físico do senhor, tá? Vou pedir. Tem alguma coisa que melhore essa dor do senhor? Alguma posição ou não tem melhora? Nada. Não tem melhora, né. Ok, vou precisar que o senhor se dirija para a maca, tá? Tá bom. Para eu iniciar o exame físico. Podem realizar, seu paciente já está na maca. Ok. Eu vou lavar minhas mãos, calçar as luvas e me paramentar para iniciar o exame físico. Considere-se paramentado. Ok. Vou pegar o estetoscópio. Primeira coisa que eu quero ver no paciente é a pressão arterial desse paciente. Consigo observar a pressão arterial? 150 por 100. 150 por 100, né. Sua pressão tá um pouquinho alta, viu, Seu Antônio? Eu quero ver agora. Vou morrer, vou morrer. Eu quero que o senhor se acalme, eu quero ver. Vou pedir para o paciente tirar a camisa, abrir a camisa, e aí eu quero ver a questão dos sinais vitais do paciente. Quero ver frequência cardíaca. 105 batimentos por minuto. De imediato também quero ver a ausculta cardíaca nos quatro pontos. Doutor, tá doendo demais, tá doendo muito. Doutor, passa alguma coisa. Regular em dois tempos, bulhas normofonéticas. Ok. Quero ver a frequência respiratória e a ausculta respiratória em ambos os campos pulmonares. Frequência respiratória de 23 e ausculta com murmúrio vesicular universalmente audível sem ruídos adventícios. Ok, saturação de oxigênio, quero ver a saturação. 89%. 89. Temperatura axilar. 37 graus. 37. Ok, o senhor não tá com febre. A ectoscopia geral do paciente, quero ver como é que o paciente se encontra. Face sudoreica. Doutor, tá doendo muito. Ok. Eu vou precisar rodar um eletro nesse paciente de doze derivações. A gente tem disponível? Tem sim. Seu eletro é esse aqui. OK. Observa-se supra de ST em derivações D2, D3 e aVF. Ok. Eu vou precisar rodar V3R e V4R. A gente tem como fazer? Para excluir infarto de ventrículo direito. Tem sim. Seu eletro é esse aqui, doutor. Ok. Normal, beleza. Vou precisar pedir para esse paciente. Seu Antônio, a gente viu aqui no seu eletro, o senhor tem uma alteraçãozinha sugestiva de infarto, viu. O senhor teve um infarto. Eu vou precisar solicitar enzimas cardíacas. Vou precisar de CPK, CKMB, troponina desse paciente. Vou precisar de hemograma completo, eletrólitos (sódio e potássio) e glicemia capilar. Vou morrer. A gente vai solicitar os exames, doutor. Ok. Aguardando resultado, não é isso? Certo. Agora então eu vou fechar. Glicemia capilar, doutor, aqui a gente tem como fazer rapidinho: 78. Normal a glicemia. Tô muito cansado, doutor. Não tô conseguindo respirar. Eu vou pedir para esse paciente, vou colocar ele em modo de improvisação. Vou pedir um acesso venoso e vou colocar oxigênio com máscara com reservatório a 10 litros por minuto. Ok. Considere a frente instalada. Ok. Ok. E aí eu vou iniciar o protocolo de infarto. Eu quero fazer nesse paciente, para alívio da dor, quero fazer morfina. Vou fazer 4 mg. São 10 mg na ampola, eu vou fazer 4 mg EV. A morfina, o oxigênio 10 litros eu coloquei. Quero fazer o nitrato, que é o isordil sublingual, 5 mg. Quero fazer pronto o AAS. Quero fazer o AAS, 300 mg comprimido para ele mastigar. Quero fazer um beta-bloqueador, carvedilol, e o clopidogrel. Vou precisar fazer o clopidogrel também, 300 mg. Vou precisar fazer a heparina nesse paciente, heparina de baixo peso molecular, e uma estatina, atorvastatina. Vou fazer nesse paciente. OK, aí eu fiz o protocolo de infarto. Agora eu quero ver se a gente tem disponível para esse paciente ser transferido para uma unidade coronariana para revascularização. A gente tem disponível? Tem sim, mas vai demorar 4 horas. 4 horas, então a gente foge da janela para revascularização. A gente tem disponível alteplase para esse paciente? Temos sim, doutor. Ok, eu vou fazer alteplase. Quero fazer. Tá doendo, doutor, por favor, me ajude. Ok, vou fazer. Qual o peso do paciente? Tenho 100 quilos, doutor. Certo, eu quero fazer 0,9 mg/kg com dose máxima de 90. Vou fazer 10% em bolus agora e os outros 90% vou fazer em bomba de infusão. Ok. Me ajude, doutor. Melhorou da dor, Seu Antônio? Que coisa boa, melhorou um bocado agora, viu. Aliviou, né. Eu vou precisar rodar outro eletro para ver se normalizou a questão do supra. Ok. Seu eletro é esse aí. Ok, estabilizou, tá sem supra já. Vou indicar a internação desse paciente, vou indicar a internação em UTI. Vou transferir esse paciente para uma UTI coronariana com avaliação do cardiologista para seguimento do caso. Vou transferir para avaliação do cardiologista. Ok, você quer entrar em contato? Eu quero contactar para ver se a gente tem como disponibilizar e transferir esse paciente com o SAMU. Tem como sim, doutor. Tem vaga na UTI coronariana do hospital de referência. Ok, então o senhor vai precisar ser transferido, tá, Seu Antônio? Você vai ser transferido e a gente vai dar seguimento ao seu caso. O senhor teve um infarto, tá bem, só que a gente agiu no tempo adequado e conseguiu salvar a vida do senhor, tudo bem? Obrigado. Ok, acho que eu encerrei minha estação. Quer encerrar? Encerrar. Ok, pode se retirar, por favor. OK, obrigado. Obrigado, pessoal.",
 "https://www.youtube.com/watch?v=zYqiGst4yr0": "Maria Boa tarde tudo bem eu sou alessandra eu vou atender a senhora hoje Pode ficar à vontade pode sentar como que a senhora ta dona maria. Tô bem graças a Deus vou passar um alquin gel aqui antes de começar o atendimento Dona Maria quem que trouxe a senhora aqui hoje eu perdi minha carteirinha de vacinação e daí só tem a da gripe do ano passado e agora não sei o que que faz e a senhora lembra se tomou alguma vacina Ah tomar sempre também mas não sei qual mas perdeu a carteirinha né então a gente vai ter que preencher essa carteirinha vai ter que tomar umas quantas vacinas aqui para deixar tudo em dia tá mas eu vou explicar certinho para senhora quando que tem que tomar qual que tem que tomar e a gente vai vai fazer tudo certinho vai ficar tudo certo não se preocupe antes eu preciso fazer umas perguntinhas para senhora aqui só para colocar no prontuário ficar atualizado Mas é bem rápido sabe aqui a senhora tem 55 anos então a senhora mora aqui perto da unidade mesmo Sim bem pertinho aqui. e a senhora mora sozinha eu e meu marido e aqui acho que a maioria das casas tem saneamento coleta tá funcionando tudo certinho sim o caminhão do lixo passa duas vezes na semana então tá ótimo e a senhora é aposentada então sim sim e o seu marido também E aí vocês fazem algum outro trabalho alguma fonte Extra de renda não não a gente vive dos aposentos dois aposentos né e consegue passar o mês se virar É apertado assim segurando as coisas tão cara né com a pandemia aumentou bastante mas a gente aprendeu a se virar né', consegui e a senhora estudou Dona Maria só até a quarta série casei nova tive os filhos cedo daí não deu para continuar Antigamente era assim né normal né hoje muito que é que os filhos estudem né mas é assim mesmo e a senhora tem alguma religião eu sou católica católica a frequência é a igreja aqui da comunidade sim todo sábado a gente va Ai que legal e a senhora tem costume de tomar algum remédio caseiro algum cházinho alguma coisa assim não sei sempre é bom né Eu sempre falo para os meus pacientes tá mal tá com alguma coisa vem aqui consultas eles precisam um remedinho Mas pode tomar um chazinho igual né sempre ajuda só não pode deixar de vir aqui e consultar como tá se sentindo bem e a senhora tem filho que a senhora falou que mora só com o seu marido eu tenho três filhos mas eles moram tudo fora ja ah não moram aqui na cidade mania de estudar né sim mas é vem visitar a senhora vem vem uma vez por mês que bom fequente então né comer a comidinha da mãe E falando nisso é a senhora que cozinha em casa sim e me conta um pouquinho como que é a alimentação tua e do teu esposo Ah mas acho que é o normal né o feijão arroz macarrão polenta Ai que delícia e salada fruta essas coisas vocês consomem salada a gente come bastante fruta que é mais de vez em quando de vez enquanto  mas tá bom é bom é importante consumir né mas a gente sabe também que é Cara essas coisas né mas e a senhora costuma fazer muita fritura sim ou não não a gente cuidar para não dar colesterol pressao alta ah tá de parabéns dona Maria é isso mesmo Tem que evitar fritura e sal na comida senhora cuida sempre acerta entao e a senhora então ela fica em casa durante o dia tem os seus afazeres né que é bastante coisa mas a senhora tira um tempinho para fazer uma atividade física Ah umas duas vezes na semana eu dou umas caminhada com as vizinhas Mas que coisa boa Então vamos começar a fazer três vezes por semana Quanto tempo mais ou menos seis caminhos meia hora 40 minutos mas 100% então é isso é muito bom e é importante fazer tá bom e a senhora bebe e alguma coisa não fuma nao Graças a Deus não tranquilo então essas perguntas a gente tem que fazer para todos os pacientes tá normal mas então agora tá tudo em dia agora vai explicar pra senhora das suas vacinas tá significa que você não já fez a da gripe no passado Você não tem a do covid. nao fiz fiquei com medo Ah é normal né Tem muita gente falando das vacinas Mas ó confia em mim que eu tô te falando é muito segura é eficaz não dá problema nenhum e a gente precisa tomar para não acontecer ter um caso gave de covid né Deus do céu então assim eu indico qua a senhora tome a do covid pode ficar tranquilo aí até aqui eles estão tendo tá tendo a vacinação no ginásio Hugo zene e aí amanhã no sábado eles vão distribuir senha bem cedinho se a senhora ir lá bem cedo eu acho que você consegue tomar já tá bom aí Eu acho melhor você fazer primeiro a do covid pela sua idade e pela questão da pandemia né Para alguma prevenção para depois se preocupar com essas outras aqui que a senhora tem que tomar. e a senhora vai lá amanhã dependendo a marca digamos assim da vacina que eles tiverem ainda do lá amanhã vai dar uma diferença de intervalo da primeira para segunda dose então o ideal é que a senhora volta que 15 dias depois da segunda dose para a gente começar a fazer essas outras vacinas tá mesmo assim eu vou te falar quais vacinas e você vai ter que tomar tá bom vai ter que ser três doses da Hepatite B você toma uma primeira dose espera um mês tomar a segunda dose e daí dá a diferença de seis meses a primeira para terceira dose aí você vai tomar aquela do tétano sabe que é bem conhecida sim três doses também toma a primeira espera dois meses para uma segunda e dentre a primeira e a terceira dose que espera quatro meses aí depois essa aí você tem que fazer cada 10 anos tá aí você tem que tomar uma dose de hepatite A uma dose de febre amarela e uma dose da tríplice viral e de novo essa da gripe que você tomou o ano passado que é da influenza né que fala e você tem que tomar todo ano tá então assim quando você voltar aqui depois a do covid a gente vai fazer o esquema vacinal bem certinho com a profissional que é responsável pelas imunizações e vai anotar as datas certinhas de qual você vai tomar Tá mas não esquece do covid que é muito importante tá bom e assim não precisa ficar preocupada porque eu vou pedir para a gente de saúde que passa lá na sua casa para ela ir se alertando e Cuidando com a senhora com a senhora não esquecer né Porque eu ando meio esquecida é normal mas aí a senhora ela te ajuda a senhora não esquece de tomar nenhuma dose tá bom eu vou vou comunicar aí assim é aqui eu sempre costumo mostrar pro meus pacientes às vezes quando não são acostumado a tomar vacina Como que é o procedimento que a senhora vai ter que tomar um monte né você viu mas não precisa ficar com medo é até engraçado aqui ó sempre tem uma frutinha para demonstrar uma seringa para ver que a agulha não é grossa não machuca e assim como é uma injeção é uma vacina é sempre a gente aplica no braço é que a gente tem um músculo que é chamado deltoide 3 4 cm abaixo desse osso que é o acrômio a gente vai fazer aplicação sempre profissional antes de fazer a vacina vai higienizar as mãos corretamente vai higienizar a tua pele também bem tranquilo para aplicar ela a vacina a gente faz uma aplicação chamada intramuscular Então ela é 90 graus a gente deixa da tua pele né é bem tranquilo é no deltoide não pode ser uma quantidade maior que um ml Então vai ser sempre meio ou um ml geralmente vacina a gente coloca assim ó bem tranquilo depois que a gente limpou a gente coloca agulha colocar o líquido tira rapidinho ou algodãozinho para estancar o sangue e passou coloca 'um curativo alguma coisa assim então não precisa ter medo que é um monte de vacina que a senhora vai ter que tomar tá porque é bem tranquilo o procedimento a seringa é descartável depois descarta a do covid também é desse jeito pode ficar tranquila que é seguro a vacina é importante tá bom a senhora tem mais alguma dúvida alguma coisa tudo certo Tudo certo certinho mesmo porque ó Sempre que precisar procura a gente tá mas eu vou falar para agente de saúde lá também passar e ver como que é Lembrar sempre da carteirinha da senhora para a senhora me fazer essas vacinas tá bom e o seu marido tomou do covid ele tomou Ah que bom então a senhora amanhã cedo vai lá eu passo o endereço certinho depois peço para as meninas passarem para você e aí amanhã você pega a senha e já vê tem essa vacina tá bom tá bom então é isso obrigado. Por nada",
  "https://www.youtube.com/watch?v=fyL7JTXQTWU": "Olá, tudo bem? Meu nome é Thiago, eu sou acadêmico aqui da faculdade de medicina e hoje eu vou produzir seu atendimento. Qual que é seu nome? Meu nome é Joana. Bom, Joana, o que está acontecendo com você aqui? Como é que eu posso te ajudar? A doutor, eu tô sentindo assim uma dor no peito, um desconforto, sabe, o meu peito ele faz uns barulhos, sabe, estranho, e esses sintomas ficam o dia todo. Como é que é? Quando que desaparece? A doutor, assim, é mais durante o dia, mas não é toda vez, não chega a despertar à noite, com essa falta de ar, com essa dor no peito, chega a acordar de noite, despertar? Ah, doutor, uma vez por mês aconteceu isso, sim, mas foi só uma vez, foi só uma vez. Ele chega a limitar suas atividades diárias, por exemplo, o exercício, arrumar a casa, sair de casa, andar? Chega atrapalhar suas atividades diárias? Ah, não, não chega a atrapalhar não, eu consigo fazer, sabe, me atrapalho não, eu só percebi que atrapalha quando eu fiz esteira, sabe, quando eu tava correndo na esteira, aí eu senti que atrapalhou, eu senti essa dor no peito parecendo um gato, sabe, miando, o chiadinho, tipo um chiado, é um gatinho, sabe, aí eu senti isso. Você já tinha sentido isso alguma outra época da sua vida, alguma coisa? Doutor, eu senti isso uma vez quando era criança, mas aí precisou fazer alguma coisa, teve que realizar algum exame, alguma coisa, cê se lembra? Eu lembro que eu fiz um exame, mas assim, não tratou. E chegou a precisar usar aquela bombinha? Você lembra? Essa bombinha eu mal usei, é. Eu vou precisar examinar você agora, tá de acordo? Vamos lá, sim, sim, vamos examinar. Vamos lá, beleza. Gente, então agora a gente vai começar com o exame físico. Por motivos de prova aqui, a gente tá usando a paciente com a blusa, né, mas geralmente é sempre indicado fazer com o paciente despido. Uma boa iluminação. Então a gente tem que começar pela inspeção pulmonar, de acordo com esse caso que a gente tá pensando, com um diagnóstico de asma. Geralmente não vai ter muitas alterações, mas é sempre bom você fazer a inspeção pulmonar, tanto anterior quanto posterior, né. Depois a gente segue com a percussão. Então a gente vai pedir para o paciente falar a palavra 33, fale 33, com isso a gente avalia a frequência vocal. Pela palpação, a gente também pode ver expansibilidade, então pede para o paciente inspirar e a gente observa o movimento dos dedos, pode inspirar só. Podemos esperar que a respiração está preservada. A gente pode realizar a percussão também, agora com a paciente com blusa não fica muito interessante. A gente realiza também a percussão nas bases. A gente realiza a percussão e o mais importante, que é no nosso quadro de asma, que é a ausculta. A gente vai principalmente buscar sons de sibilo e a gente pode escutar eles geralmente em diversos tórax, toda região posterior. É lembrando que ela escuta também, ela inclui os sons traqueais, inclui os outros sons respiratórios, mas para motivo de prova aqui, a gente vai realizar ausculta apenas no posterior do busto, tá. A gente vai realizar ausculta seguindo estes pontos e a gente vai observar principalmente o sibilos. E o sibilo são principalmente expiratórios. Lembrando que não é correto fazer avaliação com a blusa, né, mas por motivos aqui de prova, a gente tá. Às vezes num ambiente de prova não dá tempo de você pedir paciente se despir, então a gente está realizando desta maneira, mais para a fonte de avaliação. Muito obrigado, a partir de tudo que você disse, a partir dos dados que eu curei com você da história, a partir do exame físico a gente já realizou, você aparentemente tem um quadro asmático. Ele é um quadro leve, a partir do que você tem, poucos sintomas, poucos despertares noturnos, não limita muito suas atividades, os sintomas são praticamente, a maioria deles, diurnos, não usa a medicação de resgate, né. Então a partir disso a gente vai precisar solicitar um exame agora que chama espirometria, a gente avaliar o grau dessa asma, trabalhar mais corretamente essa asma se ela tem resposta ao broncodilatador, diferenciar de outras patologias. Tá bom, tá, então eu vou solicitar para você, vamos agora a gente vai avaliar sua espirometria para classificar corretamente a sua asma. Tá bom, olha, olhando os valores expiratórios respiratórios e resposta ao broncodilatador, você tem sim um diagnóstico de asma. Tá bom, e esse é um diagnóstico de asma leve. Então a gente vai precisar fazer um tratamento, né. Ele é um tratamento com corticoide inalatório, que a gente fala é a bombinha famosa, bombinha, e que usar essa bombinha já, já eu vou ensinar você corretamente como a gente vai usar e a gente vai permanecer nesse tratamento sempre que necessário. Como seu quadro é leve, quando você tiver com sintomas, você faça o uso correto dessa medicação, e lembrar também das atividades não farmacológicas que a gente tem que realizar na sua casa, então, por exemplo, diminuir a exposição à poeira, deixar a casa bastante ventilada, pegar sol, não pode deixar muito fechado, né, todos esses cuidados que tem que ter, por exemplo, com lençol antialérgico, todas as coisas para evitar também que você tenha agravamento da sua crise. Tá, e além disso, como seu quadro é leve, a gente vai poder acompanhar ambulatorialmente. Tá bom, obrigada. De nada. E aí, gostou do conteúdo? Então clica no primeiro link da nossa descrição e lá vai ter um e-book",
 "https://www.youtube.com/watch?v=6WIMmzb913E": "Oi, boa tarde, meu nome é Ana Clara, eu vou te atender hoje. Qual que é o seu nome? João. João, quantos anos você tem? 16 anos. O que está acontecendo? Como que eu posso te ajudar? Tô com muita dor na barriga aqui, ó, dor aí embaixo, à direita, isso aqui, bem embaixo. E como que começou essa dor? Faz... Ah, começou ontem à noite, mais ou menos umas 10 horas. E é no lado direito? E como que você descreveria ela para mim? É uma dor em pontada, tá. Tem algum fator que melhora essa dor para você? Não, tentei tomar remédio para gases, achei que era gases, mas não melhorou. E tem algum fator que piora essa dor? Quando eu como dá muito vômito e acaba doendo bastante também. Irradia para algum lugar? Ela caminha para algum lugar? Não, mas ela começou diferente, não. Ela começou aqui, mas em cima, no umbigo. Isso. E aí depois ela foi para cá, assim. Aí ela começou aqui, não tá conseguindo comer não, tudo que eu como eu vomito, eu não tô conseguindo comer, perdi totalmente a fome. Teve febre, minha mãe mediu ontem, deu 38 graus de febre. Tem alguma doença crônica? Não, não que eu saiba. Não toma nenhum remédio? Não tem alergia a alguma coisa? Não. Vamos ali na marcar para a gente poder te examinar, João, vou pedir para você levantar a sua blusa fazendo favor para a gente poder examinar sua barriga, certo. Então, a inspeção a gente não observa nenhuma abalamento, retração, cicatrizes, outros sinais evidentes de alguma hemorragia, a ausculta eu vou escutar os quatro quadrantes, licença, ruídos e groetes presentes nos quatro quadrantes. Vou passar para percussão, se você sentir qualquer tipo de dor você me fala, tá. Vou para apalpação, fazer a apalpação superficial primeiro, lembrando, qualquer tipo de dor que você me fala, tá. Sente alguma dor? Não. Agora vou para a versão profunda, tem alguma dor ainda? Não, aqui que dói um pouquinho. Vou fazer agora para a gente diagnosticar apendicite, a gente usa o sinal de Blueberry. Então, vou achar o ponto de Mac Burn, vou dividir da Crista ilíaca antero-superior até o umbigo, dividir em três pedaços e eu vou fazer nos dois terços inferiores. Então, é aqui, vou pedir para fazer a compressão, teve dor na descompressão? Dói. Então, não preciso repetir o exame. Vou fazer outro sinal, chamado sinal de rubizinha, em que eu vou rodar a minha mão, é do lado esquerdo, para ocorrer a movimentação de gases dentro do intestino, dói, teve dor sim. Então, é outro sinal positivo para apendicite, pode abaixar sua blusa para a gente poder voltar ali para o consultório. Bom, então, de acordo com seu exame físico, a gente já consegue fechar para você o diagnóstico de apendicite. A gente consegue excluir vários diagnósticos diferenciais só com exame físico, como por exemplo, colecistite, diverticulite, pancreatite. Então, por conta disso, a gente segue na escala de Alvarado, que é uma escala que a gente tem dentro de apendicite, a gente já consegue fechar para você direto para o tratamento, a gente não precisa pedir os exames complementares, certo. Então, eu vou chamar pessoas responsáveis para a gente poder fazer aqui a sua internação e a gente já vai solicitar a analgesia para você e vai encaminhar você direto para cirurgia, nesse caso a gente vai fazer uma apendicectomia, o nome da cirurgia, certo. Tá bom. E aí, gostou do conteúdo? Então, clica no primeiro link da nossa descrição e lá vai ter um e-book com conteúdo bem comprimido para você.",
 "https://www.youtube.com/watch?v=62oHkq8eeRc": "Boa tarde, meu nome é Ana Clara, eu vou te atender hoje, tá. Qual que é o seu nome? José. Quantos anos você tem, José? 40 anos. Como que eu posso te ajudar? Então, eu tô com uma dor bem aqui na barriga assim, ó, mas para o lado direito, perto da costela. Isso começou faz quanto tempo, José? Faz dois dias. E como que você descreveria essa dor para mim, que jeito que ela é? Então, parece uma cólica mesmo, quando tem dor de barriga assim. E ela caminha para um lugar? Ela sai daqui e vai para algum outro lugar? Não, só aqui mesmo. Tem alguma coisa que você faz que piora essa dor? Então, eu já tinha essa dor um pouquinho antes, mas era bem menos, mais para comer alguma coisa pesada assim, achava até que era estômago. E tem alguma coisa que você faz que melhora? Algum remédio? Não. Certo. Percebeu se teve febre? Tive febre. Percebeu se tá mais amarelado? Sim, também o olho, principalmente. Tá. A gente vai examinar aqui. Vamos lá na maca pra eu te examinar. Seu José, pode levantar fazendo favor a sua blusa. Vou te examinar, tá bom? Com licença. Então, a inspeção, eu não visualizo nenhum tipo de lesão, retração, abalamento, cicatrizes, nenhuma evidência de hemorragias. Vou passar para ausculta, na ausculta, eu vou examinar os quatro quadrantes, dos quatro quadrantes vou para percussão, se sentir algum tipo de dor me avisa, tudo ok, timpânico. Vou para palpação, palpação superficial, com licença, se sentir algum tipo de dor você me avisa, nenhum tipo de dor. Vou para passar uma profunda, teve dor? Não, certo. Agora, fazer um sinal que é o sinal de Murphy à procura de alguma inflamação na vesícula. O sinal de Murphy, a gente vai traçar uma linha das cicatrizes umbilicais até o rebordo costal direito e ali, nessa linha, a gente vai fazer a palpação e pedir para o paciente inspirar, por favor, inspira, expira, inspira. Ele corta a inspiração e é sinal positivo do Murphy. Pode abaixar sua blusa para a gente poder voltar ali para mesa. Então, José, de acordo com seu exame físico, a gente acaba suspeitando de algumas causas, uma delas é a colangite, uma malignidade. Então, para isso, a gente precisa pedir alguns exames, certo? Então, eu vou aqui pedir uma ultrassom de abdômen, tá, e eu vou pedir alguns exames de sangue, uma bilirrubina, Gama GT, e eu posso, vou pedir a sua internação para a gente poder fazer uma CPRE, certo, aqui os exames, obrigada. Então, José, olhando por esses exames aqui, a gente consegue fechar o seu diagnóstico, que é de colangite, certo? Então, eu vou pedir a sua internação, vou solicitar aqui a sua internação para a gente poder tratar essa colangite. Vou pedir uma para fazer analgesia para você, vou pedir para colocar você no soro para hidratar, vou pedir o antibiótico também, que é muito importante nesse caso, seu corrigir algum eletrólito que seja alterado nos seus exames, e a gente vai pedir a correção cirúrgica para você, certo? E é muito importante que a gente tome as providências agora, para a gente poder evitar que isso piore, porque a tendência é sempre isso piorar, que evolua para uma condição que chama penta de Renos, onde tem uma queda de nível de consciência, alteração de pressão, então a gente precisa tomar esses cuidados agora, certo? Tem alguma dúvida? Não. Então tá bom, pode ali para a gente poder levar o senhor pra internação, certo? E aí, gostou do conteúdo? Então, clica no primeiro link da nossa descrição que lá vai ter um e-book com conteúdo bem comprimido para você."
 }

In [ ]:
df_audios = pd.DataFrame()

for url in url_text_dict:
  link = url
  ref_text = url_text_dict[url]
  full_transcript, audio_length = transcrever_calcular(ref_text)

  df_audios = pd.concat([
      df_audios,
      pd.DataFrame({
          'link': [link],
          'audio_length': [audio_length],
          'ref_text': [ref_text],
          'full_transcript': [full_transcript]
          })
      ])

df_audios

,link,audio_length,ref_text,full_transcript
0,https://www.youtube.com/watch?v=-BsAJyoj_Kw,173035,"Boa tarde, Seu Carlos. Boa tarde, como vai? Be...",Boa tarde Seu Carlos quem vai sim seu carro me...
0,https://www.youtube.com/watch?v=xNnPFvhLm6I,614354,Bom dia. Bom dia. Bom dia. Prévia para a câmer...,Não entendi Se eu tenho 10 minutos para realiz...
0,https://www.youtube.com/watch?v=zYqiGst4yr0,495421,Maria Boa tarde tudo bem eu sou alessandra eu ...,Maria Boa tarde tudo bem eu sou Alexandre eu v...
0,https://www.youtube.com/watch?v=fyL7JTXQTWU,397758,"Olá, tudo bem? Meu nome é Thiago, eu sou acadê...",Olá tudo bem Meu nome é Thiago tá eu sou acadê...
0,https://www.youtube.com/watch?v=6WIMmzb913E,342123,"Oi, boa tarde, meu nome é Ana Clara, eu vou te...",Oi boa tarde meu nome é Ana Clara eu vou te at...
0,https://www.youtube.com/watch?v=62oHkq8eeRc,353222,"Boa tarde, meu nome é Ana Clara, eu vou te ate...",Boa tarde meu nome é Ana Clara eu vou te atend...


In [ ]:
transforms = jiwer.Compose(
    [
        # jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
  )

In [ ]:
wer_list = []
for index, row in df_audios.iterrows():
  wer = jiwer.wer(
                row['ref_text'],
                row['full_transcript'],
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )
  wer_list.append(wer)

df_audios['wer_value'] = wer_list
df_audios['acc'] = 1-df_audios['wer_value']
df_audios

,link,audio_length,ref_text,full_transcript,wer_value,acc,wer_value2
0,https://www.youtube.com/watch?v=-BsAJyoj_Kw,173035,"Boa tarde, Seu Carlos. Boa tarde, como vai? Be...",Boa tarde Seu Carlos quem vai sim seu carro me...,0.411483,0.588517,0.138211
0,https://www.youtube.com/watch?v=xNnPFvhLm6I,614354,Bom dia. Bom dia. Bom dia. Prévia para a câmer...,Não entendi Se eu tenho 10 minutos para realiz...,0.337490,0.662510,0.138211
0,https://www.youtube.com/watch?v=zYqiGst4yr0,495421,Maria Boa tarde tudo bem eu sou alessandra eu ...,Maria Boa tarde tudo bem eu sou Alexandre eu v...,0.094399,0.905601,0.138211
0,https://www.youtube.com/watch?v=fyL7JTXQTWU,397758,"Olá, tudo bem? Meu nome é Thiago, eu sou acadê...",Olá tudo bem Meu nome é Thiago tá eu sou acadê...,0.121951,0.878049,0.138211
0,https://www.youtube.com/watch?v=6WIMmzb913E,342123,"Oi, boa tarde, meu nome é Ana Clara, eu vou te...",Oi boa tarde meu nome é Ana Clara eu vou te at...,0.096506,0.903494,0.138211
0,https://www.youtube.com/watch?v=62oHkq8eeRc,353222,"Boa tarde, meu nome é Ana Clara, eu vou te ate...",Boa tarde meu nome é Ana Clara eu vou te atend...,0.138211,0.861789,0.138211


In [ ]:
jiwer.wer(
                str(df_audios['ref_text'].values),
                str(df_audios['full_transcript'].values),
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )

0.18590567076527803

In [ ]:
jiwer.wer(
                str(df_audios['ref_text'].iloc[2:].values),
                str(df_audios['full_transcript'].iloc[2:].values),
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )

0.10885805763073639

In [ ]:
df_audios.audio_length.sum() # milissegundos (~40min)

2375913

In [ ]:
df_audios.to_csv('df_audios.csv', index=False)

# LLM

In [ ]:
!export OPENAI_API_KEY="APIKEY"

os.environ["OPENAI_API_KEY"] = "APIKEY"

llm = ChatOpenAI()

In [ ]:
classificacao_dor = '''Nível de dor:
 Dor leve discreta: nota 1 (atividades normais)
 Dor leve discreta: nota 2 à 4 (Pouco Impacto nas atividades ou Faz a maioria das coisas)
 Moderada/significativa mas suportável/relativamente ruim: Nota 5 à 7 (causa dificuldade ou interrompe certas atividades)
 Grave/Muito ruim: Nota 8 (Incapacitante ou Interrompe atividades Normais)
 Intensa/Insuportável/Dilacerante: Nota 9 (Incapacitante ou Interrompe atividades Normais)
 A Pior de Todas: Nota 10 (Incapacitante ou Interrompe atividades Normais)'''

classificacao_nota_dor = '''Nível de dor:
 nota 1: Dor leve discreta
 nota 2 à 4: Dor leve discreta
 Nota 5 à 7: Moderada/significativa mas suportável/relativamente ruim
 Nota 8: Grave/Muito ruim
 Nota 9: Intensa/Insuportável/Dilacerante
 Nota 10: A Pior de Todas'''

classificacao_febre = '''
Temperatura Adultos e Crianças:
temperatura  <= 35 - hipotermia
temperatura >= 41 - Muito Quente
temperatura de 38,5 à 40,9 - Quente
temperatura de 37,5 à 38,4 - Febril
 '''

classificacao_evolucao_sinais = '''
tempo de início de evolução dos sinais:
- repentino/abrupto	: minutos/segundos
- súbito 	<= 12 horas
- Agudo	<= 24h
- Recente	< 7 dias
- Não Recente	>= 7 dias
'''

classificacao_saturacao = '''
saturação de O:
- baixa	< 95%
- muito baixa	< 92%
'''

classificacao_fc = '''
FC acordado Adulto:
- muito baixa	< 50
- alta	> 100
- muito alta	> 140
'''

classificacao_nivel_consciencia = '''
Níveis de conciência são normais a partir de 15
'''

In [ ]:
llm = ChatOpenAI()

contexto = 'considere os Fluxogramas disponíveis do protocolo machester: Dor de garganta no adulto, Dor Lombar, Dispnéia na criança, Convulsão, Dor torácica, Cefaléia'
queixa_inicial = 'Mãe entra na triagem com a criança de 1 ano convulsionando.'

fluxograma = llm.invoke(f""" {contexto}
Responda apenas com o nome do Fluxograma: A qual fluxogrma do protocolo manchester pertence,  {queixa_inicial}""")

fluxograma = fluxograma.content
print(fluxograma)

saturacao = ''
nota_dor = ''
FC = ''
temperatura = ''
nivel_consciencia = ''

sinais_vitais = {'saturacao': saturacao, 'nota_dor': nota_dor, 'FC': FC, 'temperatura': temperatura, 'nivel_consciencia': nivel_consciencia}

fluxogra_dict = dict()
fluxogra_dict[fluxograma] = {'contexto': f'''{classificacao_nota_dor}\n{classificacao_febre}\n{classificacao_evolucao_sinais}''',
'vermelho': 'Obstrução de vias aéreas/Respiração inadequada/Convulsionando(Paciente que está apresentando movimentos lônicos ou clônicos de uma crise convulsiva tipo grande mal ou apresentando convulsão parcial)/Choque/Criança não reativa/Hipoglicemia(Glicemia menor que 55 mg/dl)',
'laranja': 'Alteração do nível de consciência/Novo déficit neurológico há menos de 24h (Qualquer nova perda de função neurológica que ocorreu nas últimas 24h: alteração ou perda de sensibilidade, fraqueza de membros (transitória ou permanente), retenção urinária ou alteração da função intestinal)/Sinais de meningismo(Classicamente rigidez de nuca associada a cefaleia e fotofobia. Em crianças pequenas pode haver irritabilidade à manipulação, choro estridente e abaulamento de fontanela.)/ Erupção cutânea fixa (Erupção que não se empalidece (se lorna brancal quando pressão e aplicada sobre ela. Pode ser lestada usando-se um copo para pressionar a lesão. Não se observa alteração da coloração através do fundo do copo.) /Púrpura(Erupção de qualquer parte do corpo que é causada por pequenas hemorragias debaixo da pele. Erupção purpúrica não empalidece quando pressão é aplicada sobre ela.)/História de overdose ou envenenamento (Esta informação pode vir de outros ou pode ser deduzida na presença de caixas de remédios vazias, entre outras)/Neonato quente/Bebê quente/Criança muito quente/Adulto muito quente/Sepse possível',
'amarelo': 'Novo déficit neurológico há mais de 24h (Qualquer nova perda de função neurológica: alteração ou perda de sensibilidade, fraqueza de membros (iransitória ou permanente)retenção urinária ou alteração da função intestinal há mais de 24h)/História de trauma cranioencefálico(História de trauma físico envolvendo a cabeça. Geralmente isso será relatado pelo paciente, mas se ele esteve inconsciente, a história deverá ser colhida de uma testemunha confiável)/História discordante (Quando a história fornecida não explica os achados físicos. Pode ser um marcador de lesão não acidental em crianças ou adultos vulneráveis, podendo ser sentinela de abuso e maus tratos.)/Neonato febril/ Criança quente /Adulto quente',
'verde': 'Cefaleia (Qualquer dor ao redor da cabeça não relacionada a alguma estrutura anatômica especial. A dor facial não está incluida)/ Bebê febril/Criança febril/ Adulto febril/ Dor leve recente/ Evento recente'}

classificacao = llm.invoke(f"""
                        Considere os seguintes valores de referência: {fluxogra_dict[fluxograma]['contexto']}
                        Para o paciente com a seguinte queixa inicial: {queixa_inicial}
                        O paciente possui as seguintes informações de dados vitas: {''.join([f' {s}: {sinais_vitais[s]}' if len(str(sinais_vitais[s])) > 0 else '' for s in sinais_vitais])}

                        Avalie as perguntas em ordem e Retorne apenas o nome da cor como saída!
                        1. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['vermelho']} se sim, retorne apenas a palavra 'Vermelho', se não, siga para a próxima pergunta.
                        2. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['laranja']} se sim, retorne apenas a palavra  'Laranja', se não, siga para a próxima pergunta.
                        3. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['amarelo']} se sim, retorne apenas a palavra  'Amarelo', se não, siga para a próxima pergunta.
                        4. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['verde']} se sim, retorne apenas a palavra  'Verde', se não, siga para a próxima pergunta.
                        5. Se a resposta foi não para todas as perguntas anteriores, retorne apenas a palavra  'Azul'.


                        """)

print(classificacao.content)

Convulsão
Vermelho


In [ ]:
llm = ChatOpenAI()

queixa_inicial = 'Mulher de 33 anos com dor de cabeça, tontura e história de desmaio hoje.'

fluxograma = llm.invoke(f""" {contexto}
Responda apenas com o nome do Fluxograma: A qual fluxogrma do protocolo manchester pertence,  {queixa_inicial}""")

fluxograma = fluxograma.content
print(fluxograma)

saturacao = ''
nota_dor = '5'
FC = ''
temperatura = '36.8'
nivel_consciencia = '15'

sinais_vitais = {'saturacao': saturacao, 'nota_dor': nota_dor, 'FC': FC, 'temperatura': temperatura, 'nivel_consciencia': nivel_consciencia}

fluxogra_dict = dict()
fluxogra_dict[fluxograma] = {'contexto': f'''{classificacao_nota_dor}\n{classificacao_febre}\n{classificacao_evolucao_sinais}''',
'vermelho': 'Obstrução de vias aéreas/Respiração inadequada/Convulsionando(Paciente que está apresentando movimentos lônicos ou clônicos de uma crise convulsiva tipo grande mal ou apresentando convulsão parcial)/Choque/Criança não reativa/Hipoglicemia(Glicemia menor que 55 mg/dl)',
'laranja': 'Alteração do nível de consciência/Novo déficit neurológico há menos de 24h (Qualquer nova perda de função neurológica que ocorreu nas últimas 24h: alteração ou perda de sensibilidade, fraqueza de membros (transitória ou permanente), retenção urinária ou alteração da função intestinal)/Sinais de meningismo(Classicamente rigidez de nuca associada a cefaleia e fotofobia. Em crianças pequenas pode haver irritabilidade à manipulação, choro estridente e abaulamento de fontanela.)/ Erupção cutânea fixa (Erupção que não se empalidece (se lorna brancal quando pressão e aplicada sobre ela. Pode ser lestada usando-se um copo para pressionar a lesão. Não se observa alteração da coloração através do fundo do copo.) /Púrpura(Erupção de qualquer parte do corpo que é causada por pequenas hemorragias debaixo da pele. Erupção purpúrica não empalidece quando pressão é aplicada sobre ela.)/História de overdose ou envenenamento (Esta informação pode vir de outros ou pode ser deduzida na presença de caixas de remédios vazias, entre outras)/Neonato quente/Bebê quente/Criança muito quente/Adulto muito quente/Sepse possível',
'amarelo': 'Novo déficit neurológico há mais de 24h (Qualquer nova perda de função neurológica: alteração ou perda de sensibilidade, fraqueza de membros (iransitória ou permanente)retenção urinária ou alteração da função intestinal há mais de 24h)/História de trauma cranioencefálico(História de trauma físico envolvendo a cabeça. Geralmente isso será relatado pelo paciente, mas se ele esteve inconsciente, a história deverá ser colhida de uma testemunha confiável)/História discordante (Quando a história fornecida não explica os achados físicos. Pode ser um marcador de lesão não acidental em crianças ou adultos vulneráveis, podendo ser sentinela de abuso e maus tratos.)/Neonato febril/ Criança quente /Adulto quente',
'verde': 'Cefaleia (Qualquer dor ao redor da cabeça não relacionada a alguma estrutura anatômica especial. A dor facial não está incluida)/ Bebê febril/Criança febril/ Adulto febril/ Dor leve recente/ Evento recente'}

classificacao = llm.invoke(f"""
                        Considere os seguintes valores de referência: {fluxogra_dict[fluxograma]['contexto']}
                        Para o paciente com a seguinte queixa inicial: {queixa_inicial}
                        O paciente possui as seguintes informações de dados vitas: {''.join([f' {s}: {sinais_vitais[s]}' if len(str(sinais_vitais[s])) > 0 else '' for s in sinais_vitais])}

                        Avalie as perguntas em ordem e Retorne apenas o nome da cor como saída!
                        1. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['vermelho']} se sim, retorne apenas a palavra 'Vermelho', se não, siga para a próxima pergunta.
                        2. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['laranja']} se sim, retorne apenas a palavra  'Laranja', se não, siga para a próxima pergunta.
                        3. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['amarelo']} se sim, retorne apenas a palavra  'Amarelo', se não, siga para a próxima pergunta.
                        4. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['verde']} se sim, retorne apenas a palavra  'Verde', se não, siga para a próxima pergunta.
                        5. Se a resposta foi não para todas as perguntas anteriores, retorne apenas a palavra  'Azul'.


                        """)

print(classificacao.content)

Cefaléia
Amarelo


In [ ]:
llm = ChatOpenAI()

queixa_inicial = 'um paciente adulto de 30 anos procura a emergência relatando dor de garganta há 4 dias com febre'

fluxograma = llm.invoke(f""" {contexto}
Responda apenas com o nome do Fluxograma: A qual fluxogrma do protocolo manchester pertence,  {queixa_inicial}""")

fluxograma = fluxograma.content
print(fluxograma)

saturacao = ''
nota_dor = '3'
FC = ''
temperatura = '38.6'
nivel_consciencia = ''

sinais_vitais = {'saturacao': saturacao, 'nota_dor': nota_dor, 'FC': FC, 'temperatura': temperatura, 'nivel_consciencia': nivel_consciencia}

fluxogra_dict = dict()
fluxogra_dict[fluxograma] = {'contexto': f'''{classificacao_nota_dor}\n{classificacao_febre}\n{classificacao_evolucao_sinais}''',
'vermelho': 'Obstrução de vias aéreas/ Estridor (Ruído inspiatório, respiraório ou ambos, melhor escutado ao se respirar de boca aberta)/ Baba-se (Saliva saindo pela boca por incapacidade de engolir)/ Respiração inadequada/ Choque/ Criança não reativa',
'laranja': 'Alteração do nível de consciência/ Imunossupressão conhecida ou provável/ Risco especial de Infecção (exposição conhecida a patógeno de alto risco, quer em laboratório ou viagem para região de prevalência conhecida de doença infecciosa)/ Bebê quente/ Criança muito quente/ Adulto muito quente/ Sepse possível/ Dor intensa',
'amarelo': 'Início súbito (Início nas últimas 12h)/História de viagem recente (Viagem nas duas últimas semana)/ Criança quente/ Adulto quente/ Dor moderada',
'verde': 'Bebê febril/ Criança febril/ Adulto febril/ Dor leve recente/ Evento recente'}

classificacao = llm.invoke(f"""
                        Considere os seguintes valores de referência: {fluxogra_dict[fluxograma]['contexto']}
                        Para o paciente com a seguinte queixa inicial: {queixa_inicial}
                        O paciente possui as seguintes informações de dados vitas: {''.join([f' {s}: {sinais_vitais[s]}' if len(str(sinais_vitais[s])) > 0 else '' for s in sinais_vitais])}

                        Avalie as perguntas em ordem e Retorne apenas o nome da cor como saída!
                        1. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['vermelho']} se sim, retorne apenas a palavra 'Vermelho', se não, siga para a próxima pergunta.
                        2. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['laranja']} se sim, retorne apenas a palavra  'Laranja', se não, siga para a próxima pergunta.
                        3. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['amarelo']} se sim, retorne apenas a palavra  'Amarelo', se não, siga para a próxima pergunta.
                        4. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['verde']} se sim, retorne apenas a palavra  'Verde', se não, siga para a próxima pergunta.
                        5. Se a resposta foi não para todas as perguntas anteriores, retorne apenas a palavra  'Azul'.


                        """)

print(classificacao.content)

Dor de garganta no adulto.
Amarelo


In [ ]:
llm = ChatOpenAI()

queixa_inicial = 'um paciente Adulto entra na emergência andando, relatando dor forte lombar do lado direito, tipo cólica.  início repentino há uma hora.'

fluxograma = llm.invoke(f""" {contexto}
Responda apenas com o nome do Fluxograma: A qual fluxogrma do protocolo manchester pertence,  {queixa_inicial}""")

fluxograma = fluxograma.content
print(fluxograma)

saturacao = ''
nota_dor = '6'
FC = ''
temperatura = '36'
nivel_consciencia = ''

sinais_vitais = {'saturacao': saturacao, 'nota_dor': nota_dor, 'FC': FC, 'temperatura': temperatura, 'nivel_consciencia': nivel_consciencia}

fluxogra_dict = dict()
fluxogra_dict[fluxograma] = {'contexto': f'''{classificacao_nota_dor}\n{classificacao_febre}\n{classificacao_evolucao_sinais}''',
'vermelho': '''
- Dor lombar
- Obstrução de vias aéreas
- Respiração inadequada
- Choque''',
'laranja': '''
- Novo déficit neurológico há menos de 24h (explicação: qualquer nova perda de funçao neurológica que ocorreu nas últimas 24h: alteração ou perda da sensibilidade, fraqueza de membros [transitória ou permanente], retenção urinária ou alteração da funçaõ intestinal)
- Mecanismo de trauma significativo (explicação: Trauma penetrante [facada ou tiro] e trauma com alta transferência de energia como queda de altura e acidentes em vias de trânsito rápido[velocidade acima de 60 km/h] são significativos, principalmente se houve ejeção do veículo, mortes de outras vítimas ou grande deformação do veículo.)
- Criança muito quente
- Adulto muito quente
- Sepse possível
- Dor abdominal (explicação: Relato de qualquer dor sentida no abdomen. Se irradiada para o dorso pode indicar dissecção de aorta; se associada a sangramento vaginal,pode sugerir gravidez ectópica ou abortamento.)
- Dor intensa''',
'amarelo': '''
- Novo déficit neurológico há mais de 24h (explicação: qualquer nova perda de funçao neurológica que ocorreu nas últimas 24h: alteração ou perda da sensibilidade, fraqueza de membros [transitória ou permanente], retenção urinária ou alteração da funçaõ)
- Trauma direto no dorso (explicação: Relato de trauma no dorso por carga que ocorreu de cima para baixo. Por exemplo: quando se cai de pé ou quando a carga provoca inclinação para frente, para trás ou para o lado ou uma torção.)
- Incapaz de andar (explicação: É importante diferenciar o paciente que não consegue caminhar pela dor daquele que realmente não consegue andar. Só o último preenche tal critério.)
- Criança quente
- Adulto quente
- História discordante (explicação: Quando a história fornecida não explica os achados físicos. Pode ser um marcador de lesão não acidental em crianças ou adultos vulneráveis, podendo ser sentinela de abuso e maus tratos.)
- Cólica (explicação: Dores intermitentes [em ondas] que tendem a melhorar ou piorar acada 20 minutos.)
- Dor moderada''',
'verde': '''
- Criança febril
- Adulto febril
- Dor leve recente
- Evento recente'''}

classificacao = llm.invoke(f"""
                        Considere os seguintes valores de referência: {fluxogra_dict[fluxograma]['contexto']}
                        Para o paciente com a seguinte queixa inicial: {queixa_inicial}
                        O paciente possui as seguintes informações de dados vitas: {''.join([f' {s}: {sinais_vitais[s]}' if len(str(sinais_vitais[s])) > 0 else '' for s in sinais_vitais])}

                        Avalie as perguntas em ordem e retorne apenas a cor da alternativa correta
                        1. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['vermelho']} se sim, retorne apenas a palavra 'Vermelho', se não, siga para a próxima pergunta.
                        2. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['laranja']} se sim, retorne apenas a palavra  'Laranja', se não, siga para a próxima pergunta.
                        3. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['amarelo']} se sim, retorne apenas a palavra  'Amarelo', se não, siga para a próxima pergunta.
                        4. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['verde']} se sim, retorne apenas a palavra  'Verde', se não, siga para a próxima pergunta.
                        5. Se a resposta foi não para todas as perguntas anteriores, retorne apenas a palavra  'Azul'.
                        """)


print(classificacao.content)

Dor Lombar
Amarelo


In [ ]:
llm = ChatOpenAI()

queixa_inicial = 'um paciente Adulto andando, com dor de coluna lombar há 15 dias. Relata dor ao se movimentar que irradia para a perna.'

fluxograma = llm.invoke(f""" {contexto}
Responda apenas com o nome do Fluxograma: A qual fluxogrma do protocolo manchester pertence,  {queixa_inicial}""")

fluxograma = fluxograma.content
print(fluxograma)

saturacao = ''
nota_dor = '3'
FC = ''
temperatura = ''
nivel_consciencia = ''

sinais_vitais = {'saturacao': saturacao, 'nota_dor': nota_dor, 'FC': FC, 'temperatura': temperatura, 'nivel_consciencia': nivel_consciencia}

fluxogra_dict = dict()
fluxogra_dict[fluxograma] = {'contexto': f'''{classificacao_nota_dor}\n{classificacao_febre}\n{classificacao_evolucao_sinais}''',
'vermelho': '''
- Dor lombar
- Obstrução de vias aéreas
- Respiração inadequada
- Choque''',
'laranja': '''
- Novo déficit neurológico há menos de 24h (explicação: qualquer nova perda de funçao neurológica que ocorreu nas últimas 24h: alteração ou perda da sensibilidade, fraqueza de membros [transitória ou permanente], retenção urinária ou alteração da funçaõ intestinal)
- Mecanismo de trauma significativo (explicação: Trauma penetrante [facada ou tiro] e trauma com alta transferência de energia como queda de altura e acidentes em vias de trânsito rápido[velocidade acima de 60 km/h] são significativos, principalmente se houve ejeção do veículo, mortes de outras vítimas ou grande deformação do veículo.)
- Criança muito quente
- Adulto muito quente
- Sepse possível
- Dor abdominal (explicação: Relato de qualquer dor sentida no abdomen. Se irradiada para o dorso pode indicar dissecção de aorta; se associada a sangramento vaginal,pode sugerir gravidez ectópica ou abortamento.)
- Dor intensa''',
'amarelo': '''
- Novo déficit neurológico há mais de 24h (explicação: qualquer nova perda de funçao neurológica que ocorreu nas últimas 24h: alteração ou perda da sensibilidade, fraqueza de membros [transitória ou permanente], retenção urinária ou alteração da funçaõ)
- Trauma direto no dorso (explicação: Relato de trauma no dorso por carga que ocorreu de cima para baixo. Por exemplo: quando se cai de pé ou quando a carga provoca inclinação para frente, para trás ou para o lado ou uma torção.)
- Incapaz de andar (explicação: É importante diferenciar o paciente que não consegue caminhar pela dor daquele que realmente não consegue andar. Só o último preenche tal critério.)
- Criança quente
- Adulto quente
- História discordante (explicação: Quando a história fornecida não explica os achados físicos. Pode ser um marcador de lesão não acidental em crianças ou adultos vulneráveis, podendo ser sentinela de abuso e maus tratos.)
- Cólica (explicação: Dores intermitentes [em ondas] que tendem a melhorar ou piorar acada 20 minutos.)
- Dor moderada''',
'verde': '''
- Criança febril
- Adulto febril
- Dor leve recente
- Evento recente'''}

classificacao = llm.invoke(f"""
                        Considere os seguintes valores de referência: {fluxogra_dict[fluxograma]['contexto']}
                        Para o paciente com a seguinte queixa inicial: {queixa_inicial}
                        O paciente possui as seguintes informações de dados vitas: {''.join([f' {s}: {sinais_vitais[s]}' if len(str(sinais_vitais[s])) > 0 else '' for s in sinais_vitais])}

                        Avalie as perguntas em ordem e retorne apenas a cor da alternativa correta
                        1. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['vermelho']} se sim, retorne apenas a palavra 'Vermelho', se não, siga para a próxima pergunta.
                        2. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['laranja']} se sim, retorne apenas a palavra  'Laranja', se não, siga para a próxima pergunta.
                        3. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['amarelo']} se sim, retorne apenas a palavra  'Amarelo', se não, siga para a próxima pergunta.
                        4. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['verde']} se sim, retorne apenas a palavra  'Verde', se não, siga para a próxima pergunta.
                        5. Se a resposta foi não para todas as perguntas anteriores, retorne apenas a palavra  'Azul'.
                        """)


print(classificacao.content)

Dor Lombar
Azul


In [ ]:
llm = ChatOpenAI()

queixa_inicial = 'um paciente Adulto, Homem de 67 anos procura o serviço de emergência com dor no peito tipo aperto no centro do peito de início há 2 horas.'

fluxograma = llm.invoke(f""" {contexto}
Responda apenas com o nome do Fluxograma: A qual fluxogrma do protocolo manchester pertence,  {queixa_inicial}""")

fluxograma = fluxograma.content
print(fluxograma)

saturacao = '95'
nota_dor = '5'
FC = '88'
temperatura = ''
nivel_consciencia = ''

sinais_vitais = {'saturacao': saturacao, 'nota_dor': nota_dor, 'FC': FC, 'temperatura': temperatura, 'nivel_consciencia': nivel_consciencia}

fluxogra_dict = dict()
fluxogra_dict[fluxograma] = {'contexto': f'''{classificacao_nota_dor}\n{classificacao_evolucao_sinais}\n{classificacao_saturacao}\n{classificacao_fc}''',
'vermelho': '''
- Dor lombar
- Obstrução de vias aéreas
- Respiração inadequada
- Choque''',
'laranja': '''
- Novo déficit neurológico há menos de 24h (explicação: qualquer nova perda de funçao neurológica que ocorreu nas últimas 24h: alteração ou perda da sensibilidade, fraqueza de membros [transitória ou permanente], retenção urinária ou alteração da funçaõ intestinal)
- Mecanismo de trauma significativo (explicação: Trauma penetrante [facada ou tiro] e trauma com alta transferência de energia como queda de altura e acidentes em vias de trânsito rápido[velocidade acima de 60 km/h] são significativos, principalmente se houve ejeção do veículo, mortes de outras vítimas ou grande deformação do veículo.)
- Criança muito quente
- Adulto muito quente
- Sepse possível
- Dor abdominal (explicação: Relato de qualquer dor sentida no abdomen. Se irradiada para o dorso pode indicar dissecção de aorta; se associada a sangramento vaginal,pode sugerir gravidez ectópica ou abortamento.)
- Dor intensa''',
'amarelo': '''
- Novo déficit neurológico há mais de 24h (explicação: qualquer nova perda de funçao neurológica que ocorreu nas últimas 24h: alteração ou perda da sensibilidade, fraqueza de membros [transitória ou permanente], retenção urinária ou alteração da funçaõ)
- Trauma direto no dorso (explicação: Relato de trauma no dorso por carga que ocorreu de cima para baixo. Por exemplo: quando se cai de pé ou quando a carga provoca inclinação para frente, para trás ou para o lado ou uma torção.)
- Incapaz de andar (explicação: É importante diferenciar o paciente que não consegue caminhar pela dor daquele que realmente não consegue andar. Só o último preenche tal critério.)
- Criança quente
- Adulto quente
- História discordante (explicação: Quando a história fornecida não explica os achados físicos. Pode ser um marcador de lesão não acidental em crianças ou adultos vulneráveis, podendo ser sentinela de abuso e maus tratos.)
- Cólica (explicação: Dores intermitentes [em ondas] que tendem a melhorar ou piorar acada 20 minutos.)
- Dor moderada''',
'verde': '''
- Criança febril
- Adulto febril
- Dor leve recente
- Evento recente'''}

classificacao = llm.invoke(f"""
                        Considere os seguintes valores de referência: {fluxogra_dict[fluxograma]['contexto']}
                        Para o paciente com a seguinte queixa inicial: {queixa_inicial}
                        O paciente possui as seguintes informações de dados vitas: {''.join([f' {s}: {sinais_vitais[s]}' if len(str(sinais_vitais[s])) > 0 else '' for s in sinais_vitais])}

                        Avalie as perguntas em ordem e retorne apenas a cor da alternativa correta
                        1. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['vermelho']} se sim, retorne apenas a palavra 'Vermelho', se não, siga para a próxima pergunta.
                        2. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['laranja']} se sim, retorne apenas a palavra  'Laranja', se não, siga para a próxima pergunta.
                        3. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['amarelo']} se sim, retorne apenas a palavra  'Amarelo', se não, siga para a próxima pergunta.
                        4. Ele se enquadra em pelo menos um dos seguintes requisitos? {fluxogra_dict[fluxograma]['verde']} se sim, retorne apenas a palavra  'Verde', se não, siga para a próxima pergunta.
                        5. Se a resposta foi não para todas as perguntas anteriores, retorne apenas a palavra  'Azul'.
                        """)


print(classificacao.content)

Dor torácica
Laranja
